In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from keras.callbacks import TensorBoard
import keras
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [ ]:
# Kaggle users: this is where you import the keypoints files you've generated

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# code to read desired number of keypoint data instances based on classes or videos. 55 rows per video and 550 rows per class.

start_limit = 0
end_limit = 16500

X=np.load('/content/drive/MyDrive/PslKeypoints/psl_keypoints.npy')
# X = X[start_limit:end_limit]

Y=np.load('/content/drive/MyDrive/PslKeypoints/psl_labels.npy')
# Y = Y[start_limit:end_limit]

In [ ]:
# code to skip over a few augmentations (optional)

# new_X = []
# new_Y = []

# i = 0
# while i < X.shape[0]:
#     for j in range(15):
#       # if j!=2 and j!=3 and j!=6 and j!=7:
#       #   new_X.append(X[i+j])
#       #   new_Y.append(Y[i+j])
#       new_X.append(X[i+j])
#       new_Y.append(Y[i+j])
#     i += 55

# X = np.array(new_X)
# Y = np.array(new_Y)

In [ ]:
# splitting and preparing data for training

y = to_categorical(Y).astype(int)
# y = y[:,:]
print(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
print(X.shape)
print(y.shape)


In [ ]:
del X
del Y

In [ ]:
# # Uncomment whichever model you want to use


# # LSTM model
# model = Sequential()
# model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(45,258)))
# model.add(Dropout(0.4))      ##0.4
# model.add(LSTM(256, return_sequences=True, activation='relu'))
# model.add(Dropout(0.4))      ##0.4
# model.add(LSTM(256, return_sequences=False, activation='relu'))
# model.add(Dropout(0.4))      ##//

# model.add(BatchNormalization())

# model.add(Dense(256, activation='relu'))
# model.add(Dense(256, activation='relu'))   ##// A2
# model.add(Dense(128, activation='relu'))   ##//
# model.add(Dense(64, activation='relu'))    ##//

# model.add(Dense(15, activation='softmax'))

# model.summary()




# # GRU model
# from tensorflow.keras.layers import GRU
# model = Sequential()
# model.add(GRU(128, return_sequences=True, activation='relu', input_shape=(45, 258)))
# model.add(Dropout(0.4))
# model.add(GRU(256, return_sequences=True, activation='relu'))
# model.add(Dropout(0.4))
# model.add(GRU(256, return_sequences=False, activation='relu'))
# model.add(Dropout(0.4))

# model.add(BatchNormalization())

# model.add(Dense(256, activation='relu'))
# model.add(Dense(256, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))

# model.add(Dense(15, activation='softmax'))

# model.summary()





# # Simple RNN model
# from tensorflow.keras.layers import SimpleRNN
# model = Sequential()
# model.add(SimpleRNN(128, return_sequences=True, activation='relu', input_shape=(45, 258)))
# model.add(Dropout(0.3))
# model.add(SimpleRNN(256, return_sequences=True, activation='relu'))
# model.add(Dropout(0.3))
# model.add(SimpleRNN(256, return_sequences=False, activation='relu'))
# model.add(Dropout(0.3))

# model.add(BatchNormalization())

# model.add(Dense(256, activation='relu'))
# model.add(Dense(256, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))

# model.add(Dense(15, activation='softmax'))

# model.summary()





# # Bidirectional RNN
# from tensorflow.keras.layers import Bidirectional, SimpleRNN
# model = Sequential()
# model.add(Bidirectional(SimpleRNN(128, return_sequences=True, activation='relu'), input_shape=(45, 258)))
# model.add(Dropout(0.3))
# model.add(Bidirectional(SimpleRNN(256, return_sequences=True, activation='relu')))
# model.add(Dropout(0.3))
# model.add(Bidirectional(SimpleRNN(256, return_sequences=False, activation='relu')))
# # model.add(Dropout(0.3))

# model.add(BatchNormalization())

# model.add(Dense(256, activation='relu'))
# # model.add(Dense(256, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))

# model.add(Dense(15, activation='softmax'))

# model.summary()

In [ ]:
# setting hyperparameters and training

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=600, callbacks=[callback],batch_size=256)

In [ ]:

# Increase the epoch interval
step_size = 3

# Plotting training and validation accuracy
plt.plot(range(1, len(model.history.history['categorical_accuracy']) + 1, step_size), model.history.history['categorical_accuracy'][::step_size], label="training accuracy")
plt.plot(range(1, len(model.history.history['val_categorical_accuracy']) + 1, step_size), model.history.history["val_categorical_accuracy"][::step_size], label="validation accuracy")
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Plotting training and validation loss
plt.plot(range(1, len(model.history.history['loss']) + 1, step_size), model.history.history["loss"][::step_size], label="training loss")
plt.plot(range(1, len(model.history.history['val_loss']) + 1, step_size), model.history.history["val_loss"][::step_size], label="validation loss")
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()



In [ ]:
model.save('RP_RNN_PSL_15C_45F_256B_20T_.h5')
# del model, X_train, X_test, y_train, y_test

In [ ]:
# accuracy test


yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

cm = confusion_matrix(ytrue, yhat)
# multilabel_confusion_matrix(ytrue, yhat)

# Creating a heatmap of the confusion matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')

# Adding labels and title
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')

# Displaying the plot
plt.show()

In [ ]:
accuracy_score(ytrue, yhat)